# In-Context Learning


In [1]:
import os
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai
from urllib.request import urlopen, urlretrieve
from IPython.display import Markdown, display
from pypdf import PdfReader
from datetime import date
from tqdm import tqdm

In [2]:
API_KEY = os.environ.get("GEMINI_API_KEY")
genai.configure(api_key=API_KEY)

In [3]:
BASE_URL = "https://huggingface.co/papers"
page = requests.get(BASE_URL)
soup = BeautifulSoup(page.content, "html.parser")
h3s = soup.find_all("h3")

papers = []

for h3 in h3s:
    a = h3.find("a")
    title = a.text
    link = a["href"].replace('/papers', '')

    papers.append({"title": title, "url": f"https://arxiv.org/pdf{link}"})

In [4]:
def extract_paper(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text


def extract_pdf(url):
    pdf = urlretrieve(url, "pdf_file.pdf")
    reader = PdfReader("pdf_file.pdf")
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text


def printmd(string):
    display(Markdown(string))

In [5]:
LLM = "gemini-1.5-flash"
model = genai.GenerativeModel(LLM)

In [6]:
for paper in tqdm(papers):
    try:
        paper["summary"] = model.generate_content("Summarize this research article into one paragraph without formatting highlighting its strengths and weaknesses. " + extract_pdf(paper["url"])).text
    except:
        print("Generation failed")
        paper["summary"] = "Paper not available"

100%|██████████| 18/18 [01:08<00:00,  3.80s/it]


In [7]:
page = f"<html> <head> <h1>Daily Dose of AI Research</h1> <h4>{date.today()}</h4> <p><i>Summaries generated with: {LLM}</i>"
with open("papers.html", "w") as f:
    f.write(page)
for paper in papers:
    page = f'<h2><a href="{paper["url"]}">{paper["title"]}</a></h2> <p>{paper["summary"]}</p>'
    with open("papers.html", "a") as f:
        f.write(page)
end = "</head>  </html>"
with open("papers.html", "a") as f:
    f.write(end)

In [8]:
for paper in papers:
    printmd("**[{}]({})**<br>{}<br><br>".format(paper["title"],
                                                paper["url"],
                                                paper["summary"]))

**[OS-ATLAS: A Foundation Action Model for Generalist GUI Agents](https://arxiv.org/pdf/2410.23218)**<br>This research introduces OS-ATLAS, an open-source foundational action model for generalist GUI agents. OS-ATLAS addresses the shortcomings of previous open-source models by excelling in GUI grounding and out-of-distribution (OOD) scenarios.  It achieves this through innovations in both data and modeling.  The key strength of OS-ATLAS is its extensive open-source multi-platform GUI grounding corpus, containing over 13 million GUI elements across Windows, macOS, Linux, Android, and the web. This large dataset, combined with a unified action space to address action naming conflicts, allows OS-ATLAS to understand GUI screenshots and generalize to unseen interfaces. While OS-ATLAS shows significant performance improvements over previous models, a limitation lies in its reliance on GPT-4o for certain tasks, highlighting the need for further research on fully open-source alternatives. The paper also emphasizes the importance of scaling and improving data collection methods to further enhance performance. 
<br><br>

**[Constant Acceleration Flow](https://arxiv.org/pdf/2411.00322)**<br>This research paper introduces Constant Acceleration Flow (CAF), a novel ordinary differential equation (ODE) framework for generative modeling. CAF improves upon existing rectified flow methods by incorporating acceleration as a learnable variable, enabling more precise ODE flow estimation and faster generation. To mitigate the flow crossing problem, which hinders the learning of straight ODE trajectories, the authors propose two techniques: initial velocity conditioning and a reflow procedure for the initial velocity. The authors demonstrate CAF’s effectiveness through extensive experiments on synthetic and real-world image datasets, achieving state-of-the-art Fréchet Inception Distance (FID) scores on CIFAR-10 and ImageNet 64x64. While CAF demonstrates impressive performance in terms of image quality and speed, it faces limitations such as increased computational burden due to additional function evaluations and the requirement of supplementary data for deterministic coupling. Nonetheless, the paper highlights CAF’s potential for positive societal impact across various domains, while acknowledging the crucial need for responsible development and deployment to mitigate potential risks associated with generative models. 
<br><br>

**[TOMATO: Assessing Visual Temporal Reasoning Capabilities in Multimodal Foundation Models](https://arxiv.org/pdf/2410.23266)**<br>## Analysis of Common Failure Cases in TOMATO

This document provides a detailed analysis of common failure cases across various reasoning types in TOMATO, highlighting the challenges faced by both humans and large language models (LLMs) in understanding and interpreting visual temporal information. 

**Key Observations:**

* **Human Limitations:** Human annotators often struggle with subtle visual cues, particularly in static frames. This is especially evident in tasks like rotation, direction, and velocity & frequency, where motion is not explicitly captured but must be inferred.
* **LLM Challenges:** LLMs, despite their advancements in language and visual understanding, are still susceptible to misinterpretations and biases in handling visual temporal reasoning. Their performance is heavily influenced by factors like frame quality, object clarity, and the complexity of the motion.
* **Ambiguity and Context:** Many failure cases arise from ambiguity in the provided frames or a lack of sufficient context. This can lead to multiple plausible interpretations, making it difficult for both humans and LLMs to choose the most accurate answer. 

**Specific Failure Cases:**

**Rotation:**

* **Incorrect Direction:** LLMs often misjudge the direction of rotation due to the limited visual information in the frames. This is particularly challenging when the rotation is not fully visible or when the motion is subtle.
* **Assuming Constant Speed:** LLMs may incorrectly assume constant speed, overlooking subtle changes in rotational speed that are not easily discernible in static frames.

**Direction:**

* **Difficulty with Complex Motion:** LLMs struggle with understanding and interpreting complex motion patterns, especially when the direction changes or when the movement is not linear. 
* **Over-reliance on Visual Cues:** LLMs may over-rely on visual cues and miss subtle hints about directional changes that are not explicitly shown in the frames. 

**Velocity & Frequency:**

* **Inability to Gauge Speed:** LLMs are often unable to accurately gauge the speed of objects based solely on visual information, especially when the motion is subtle or when the frame rate is low.
* **Misinterpreting Acceleration/Deceleration:** LLMs may struggle to differentiate between acceleration, deceleration, and constant speed, particularly in scenes with gradual changes in movement.

**Shape & Trend:**

* **Inferring Shape from Trajectory:** LLMs have difficulty inferring the exact shape of a trajectory based on limited visual information, often making incorrect guesses about the geometry of the path.

**Visual Cues:**

* **Difficulty with Temporal Ordering:** LLMs struggle with understanding the temporal ordering of events based on visual cues alone. They may incorrectly assume simultaneous actions or fail to identify the exact starting point of an action.

**Action Count:**

* **Counting Discrepancies:** LLMs often have difficulty counting actions accurately, especially when the frames do not provide a clear view of the full action or when the motion is continuous.

**Recommendations:**

* **Improve Frame Quality and Resolution:** Provide higher quality and higher resolution frames to minimize ambiguity and improve visual clarity for both humans and LLMs.
* **Include More Contextual Information:** Provide additional information about the scene, objects, and relevant events to aid in understanding and interpreting visual temporal reasoning.
* **Develop Novel Evaluation Metrics:** Explore novel evaluation metrics that consider the complexity of visual temporal reasoning and the potential for multiple plausible interpretations.
* **Fine-tune LLMs with Temporal Data:** Train LLMs on datasets with specific visual temporal reasoning tasks to enhance their ability to understand and interpret motion patterns.

**Conclusion:**

The common failure cases in TOMATO highlight the ongoing challenges in developing robust visual temporal reasoning systems. While significant progress has been made in understanding visual and language information, accurately interpreting complex motion patterns remains a significant challenge. By addressing the limitations discussed in this analysis, researchers can continue to advance the field of visual temporal reasoning and develop more comprehensive and reliable systems for understanding and interpreting visual information across time. 
<br><br>

**[Personalization of Large Language Models: A Survey](https://arxiv.org/pdf/2411.00027)**<br>This research article provides a comprehensive survey of personalization in large language models (LLMs). It offers a unifying taxonomy for personalized LLM usage, categorizing efforts into direct personalized text generation and downstream task personalization. The paper formalizes key concepts of personalization, defines different levels of granularity (user-level, persona-level, and global preference alignment), and provides a detailed taxonomy of techniques, including retrieval-augmented generation, prompting, representation learning, and RLHF. It also reviews evaluation methodologies, distinguishing between intrinsic and extrinsic evaluation, and categorizes datasets based on the presence or absence of user-specific ground-truth text. The article highlights the potential of personalized LLMs in various domains, including AI assistants in education and healthcare, finance, legal, and coding environments, as well as recommendation systems and search engines. However, it also acknowledges crucial challenges such as the need for improved benchmarks and metrics, tackling the cold-start problem, addressing stereotypes and biases, ensuring privacy, and expanding personalization to multimodal systems. Despite these challenges, the field of personalized LLMs is rapidly evolving, with the potential to significantly impact human-AI interaction across diverse domains. 
<br><br>

**[Randomized Autoregressive Visual Generation](https://arxiv.org/pdf/2411.00776)**<br>This research paper presents Randomized AutoRegressive (RAR) modeling, a new approach for visual generation that achieves state-of-the-art performance while maintaining compatibility with language modeling frameworks.  RAR introduces randomness annealing, where the input sequence is randomly permuted during training with a probability that decays over time, enabling the model to learn bidirectional contexts. This strategy significantly improves generation quality without compromising the autoregressive structure. RAR outperforms prior autoregressive image generators and even surpasses leading diffusion-based and masked transformer-based methods on the ImageNet-256 benchmark. However, RAR's strength in capturing "global context" during generation remains a challenge, as some tokens are generated before others, lacking complete global context. Additionally, the randomness annealing strategy might introduce sub-optimal behavior, potentially making the model focus on learning permutation orders rather than generation quality. Despite these weaknesses, RAR represents a promising advancement in autoregressive visual generation, opening possibilities for further research and development. 
<br><br>

...

**[GPT or BERT: why not both?](https://arxiv.org/pdf/2410.24159)**<br>This research paper proposes a novel method for merging masked language modeling (MLM) and causal language modeling (CLM) into a single transformer architecture, resulting in a model called GPT-BERT. This hybrid approach aims to combine the strengths of both paradigms, leading to a more general language understanding. The authors demonstrate that GPT-BERT significantly outperforms both masked-only and causal-only models on the BabyLM Challenge 2024, achieving better results across multiple benchmarks. Additionally, GPT-BERT exhibits unexpected capabilities like in-context learning despite its relatively small size. However, the research highlights the need for further investigation, as the model's effectiveness in larger-scale settings and with larger datasets remains to be explored. 
<br><br>

**[Zipfian Whitening](https://arxiv.org/pdf/2411.00680)**<br>This research paper proposes Zipfian whitening, a novel method for enhancing the symmetry of word embedding spaces by incorporating empirical word frequencies. The authors demonstrate that traditional whitening methods, which implicitly assume uniform word frequencies, are inefficient in NLP due to the highly non-uniform distribution of word frequencies, often following Zipf's law. Zipfian whitening, by weighting expected values with word frequencies, consistently outperforms baseline methods and achieves high correlation with downstream task performance. The authors explain the effectiveness of their method by framing it within the context of exponential families, highlighting how the Zipfian prior emphasizes low-frequency words, which are more informative, both in terms of vector norm and loss function. Notably, this work unifies several existing NLP methods, including word2vec, WhiteningBERT, and headless language models, under the umbrella of Zipfian priors, providing a theoretical foundation for their success. However, the research also acknowledges limitations, including potential numerical instability and the need for further exploration of higher-order moments and the relationship between whitening and generative models. Overall, the paper offers a valuable contribution to the field of word embedding, highlighting the importance of considering word frequencies and paving the way for improved NLP models. 
<br><br>

**[GRS-QA -- Graph Reasoning-Structured Question Answering Dataset](https://arxiv.org/pdf/2411.00369)**<br>The GRS-QA dataset is a novel question-answering dataset that incorporates explicit reasoning structures in the form of graphs, representing the logical steps required to answer multi-hop questions. This unique feature distinguishes it from existing multi-hop QA datasets that lack such explicit structures. GRS-QA's strength lies in its ability to provide a transparent framework for analyzing how LLMs handle complex reasoning, allowing researchers to pinpoint where models struggle at a fine-grained level. However, the dataset suffers from an imbalanced distribution of graph types, with certain reasoning structures being overrepresented while more complex ones are underrepresented. Additionally, the dataset spans multiple domains, making it difficult to evaluate domain-specific reasoning abilities. Despite these limitations, GRS-QA holds promise as a valuable resource for evaluating and improving the reasoning capabilities of LLMs. 
<br><br>

**[Physics in Next-token Prediction](https://arxiv.org/pdf/2411.00660)**<br>This research paper investigates the underlying physics of Next-token Prediction (NTP), the core mechanism driving the intelligence of autoregressive AI models. The authors propose the First Law of Information Capacity (IC-1), which states that the training process of these models is essentially a transfer of information from the dataset to the model, with the information capacity of the model increasing proportionally to the amount of information transferred.  Furthermore, they introduce Landauer's Principle into NTP and formulate the Second Law of Information Capacity (IC-2), which establishes the relationship between model training and energy consumption, quantifying the minimum energy required for information transfer. Several practical corollaries are derived from these laws, providing insights into estimating dataset entropy and quality, and matching model size with dataset size. The paper also validates the compatibility and complementarity of their findings with existing theories like the Scaling Law of Neural Language Models and Knowledge Capacity Scaling Laws. The strengths of the research lie in its innovative approach to linking the physics of information and energy to AI model training, offering a novel perspective on intelligence emergence. However, it's worth noting that some of the derived relationships are based on assumptions and simplifications, and require further empirical validation. 
<br><br>

**[WikiNER-fr-gold: A Gold-Standard NER Corpus](https://arxiv.org/pdf/2411.00030)**<br>This research article presents WikiNER-fr-gold, a gold-standard Named Entity Recognition (NER) corpus in French. The corpus is built upon a 20% random sample of the original WikiNER-fr corpus, which was annotated semi-automatically using Wikipedia hyperlinks. The authors manually corrected and standardized the annotations, addressing inconsistencies and errors arising from the semi-supervised process. While the study highlights common errors and provides correction strategies, a limitation is the lack of comparison with other annotation schemes, making it challenging to assess the effectiveness of their approach. Future work aims to expand the revision to the entire WikiNER-fr corpus, explore automation possibilities, and integrate active learning techniques. This could lead to a more comprehensive and robust gold-standard NER resource for French. 
<br><br>